# Future Sales Prediction

In [ ]:
'''

This script uses a time-series dataset consisting of daily sales of all 
products for a software company with multiple stores/branches (train set), 
evaluates several models on the train set, and uses the best model to predict 
the total sales for every product and store in the following month (test set).

'''
__author__ = "Mahsa Shokouhi"
__email__ = "mahsa_shokouhi@yahoo.com"

## Import Packages

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from math import sqrt

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from lightgbm import LGBMRegressor

%matplotlib inline

## Define Functions

In [ ]:
def get_info(df):
    ''' Overview of a dataframe '''
    print('Dimensions:', df.shape)
    print('\nThe first and last 3 lines:')

    return df.iloc[list(range(3))+list(range(-3, 0))]


def add_date_features(df):
    ''' Extract and add year, month, and season to features. '''
    df['year'] = df.date.map(lambda x: x.year)
    df['month'] = df.date.map(lambda x: x.month)
    df['season'] = ((df.month - 1) // 3) + 1

    return df


def mean_encode(df, feature, target):
    ''' Mean-encode selected feature with target: 
        add mean tartget value for each level of the feature.
        Then add them to the featues. '''
    kf = KFold(n_splits=5, random_state=None, shuffle=False)

    temp = [None] * df.shape[0]
    temp = pd.Series(temp)

    for idx_train, idx_val in kf.split(df):
        train = df.iloc[idx_train]
        validation = df.iloc[idx_val]

        means = train.groupby(feature)[target].mean()
        temp[idx_val] = validation[feature].map(means)

    df[feature + '_mean_encoded'] = temp
    mean_target = df[target].mean()
    df[feature + '_mean_encoded'].fillna(mean_target, inplace=True)

    # Correlation between mean_encoded feature and target
    encoded_feature = df[feature + '_mean_encoded'].values
    corr = np.corrcoef(df[target].values, encoded_feature)[0][1]
    print('Correlation between mean_encoded {} and target ='.format(feature), corr)

    return df


def aggregate_monthly(df, group_cols, agg_feature, agg_name, fill_na=0):
    ''' Aggregate to get total monthly resuluts for the selected feature.'''
    gb = df.groupby(group_cols, as_index=False)[
        agg_feature].agg({agg_name: sum})
    df_out = pd.merge(df, gb, how='left', on=group_cols).fillna(fill_na)

    return df_out


def add_lag_features(df, lag_cols, index_cols, max_lag):
    ''' Add data from previous months (lags) for the selected features
        to the list of features for training. '''
    temp1 = df.copy()[index_cols + lag_cols]

    for lag in np.arange(1, max_lag+1):
        temp2 = temp1.copy()
        temp2['date_block_num'] = temp2['date_block_num'] + lag
        col_name = '_lag' + str(lag)
        df = pd.merge(df, temp2, how='left', on=index_cols,
                      suffixes=('', col_name)).fillna(0)

    df.drop_duplicates(inplace=True)

    return df


def lag_cor_plot(data, col, n_lags):
    ''' Plot the changes and correlations of lagged-features (previous months) 
        with the current month values. '''
    n_rows = n_lags//3  # 3 columns
    plt.figure(figsize=(16, n_rows*4))  # hight=4 for each row
    x = col
    for i in range(1, n_lags+1):
        y = x + '_lag' + str(i)
        cor = np.corrcoef(data[x], data[y])[0, 1]

        ax = plt.subplot(n_rows, 3, i)
        ax.set(xticklabels=[])
        ax.set(yticklabels=[])
        ax.tick_params(axis=u'both', which=u'both', length=0)
        ax.set_title('Correlation = %.2f' % cor)
        sns.scatterplot(x, y, data=data, ax=ax)


def get_train_test(df, target, cut_off):
    ''' Train-test split: keep the last month for test. '''
    max_months = max(df['date_block_num'])

    # remove old data (before cut_off value)
    train = df[df['date_block_num'].between(
        cut_off, max_months-1)].drop('date_block_num', axis=1)
    test = df[df['date_block_num'] ==
                max_months].drop('date_block_num', axis=1)
    
    x_train = train.drop(target, axis=1)
    y_train = train[target]

    x_test = test.drop(target, axis=1)
    y_test = test[target]


    return x_train, y_train, x_test, y_test

## Load Data

In [ ]:
sales = pd.read_csv('sales_train.csv')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

test_data = pd.read_csv('test.csv')

## Data Wrangling

In [ ]:
# Convert dates to datetime format
sales['date'] = pd.to_datetime(sales.date, format="%d.%m.%Y")

In [ ]:
# Merge the training data
sales = pd.merge(sales, items, on='item_id',
                 how='left').drop('item_name', axis=1)
sales.describe()

In [ ]:
# Get transactions with negative 'item_price' or 'item_cnt_day'
price_negative = sales[sales['item_price'] <= 0]
count_negative = sales[sales['item_cnt_day'] < 0]

print(price_negative.shape, count_negative.shape)

In [ ]:
# Remove the only row with negative price
# Items with negative count correspond to returns

idx = price_negative.index[0]
sales = sales.drop(idx, axis=0)

# Get separate columns for sales and returns
sales['isReturned'] = (sales['item_cnt_day'] < 0).astype(int)
sales['return_cnt_day'] = abs(sales['item_cnt_day']) * sales['isReturned']
sales['sell_cnt_day'] = abs(sales['item_cnt_day']) * (sales['item_cnt_day'] > 0).astype(int)
sales = sales.drop('item_cnt_day', axis=1)

get_info(sales)

In [ ]:
# Add month, year, and season to features
sales = add_date_features(sales)


# Aggregate to get monthly data
# 1. Total monthly sale for each item at each shop
sales = aggregate_monthly(sales,
                          ['date_block_num', 'shop_id', 'item_id'],
                          'sell_cnt_day', 'monthly_sale', fill_na=0)
# 2. Total monthly returns for each item at each shop
sales = aggregate_monthly(sales,
                          ['date_block_num', 'shop_id', 'item_id'],
                          'return_cnt_day', 'monthly_return', fill_na=0)
# 3. Total monthly sale for all items at each shop
sales = aggregate_monthly(sales,
                          ['shop_id', 'date_block_num'],
                          'sell_cnt_day', 'monthly_sale_shop', fill_na=0)
# 4. Total monthly sale for each item across all shops
sales = aggregate_monthly(sales,
                          ['item_id', 'date_block_num'],
                          'sell_cnt_day', 'monthly_sale_item', fill_na=0)

# Net monthly sales
sales['total_monthly_sale'] = (
    sales['monthly_sale'] - sales['monthly_return']).clip(lower=0)


# Reorder columns, and sort
index_cols = ['shop_id', 'item_id', 'date_block_num']
cols_other = list(sales.columns.difference(index_cols))
temp = sales[index_cols + cols_other]
sales = temp


# Remove duplicate rows corresponding to daily data (different days with the 
# same monthly values). Keep the most recent row with the latest price.
sales.sort_values(index_cols + ['date'], inplace=True)
sales.drop_duplicates(subset=index_cols, keep='last', inplace=True) 
sales = sales.drop(['date', 'sell_cnt_day', 'return_cnt_day',
                  'isReturned', 'monthly_sale'], axis=1)
sales = sales.reset_index(drop=True)


get_info(sales)

## Exploratory Data Analysis

In [ ]:
# Lag plots with 12-months lags of the total_monthly_sale

data = sales.copy()

index_cols = ['shop_id', 'item_id', 'date_block_num']
lag_cols = ['total_monthly_sale']
data = add_lag_features(data, lag_cols, index_cols, 12)

lag_cor_plot(data, 'total_monthly_sale', 12)

In [ ]:
# Correlation plot

plt.figure(figsize=(15, 10))
sns.heatmap(data.corr(), xticklabels=data.columns,
            yticklabels=data.columns, cmap="RdBu_r");
plt.title('Correlaiton between variables', fontsize=20);

In [ ]:
# Correlation of features with target variable
df = pd.DataFrame(data.corr().total_monthly_sale, index=data.corr().index)
plt.figure(figsize=(4, 12))
sns.heatmap(df, annot=True, cmap="RdBu_r");
plt.title('Correlaiton with target', fontsize=20);

## Baseline model
Monthly sale for each item, at each shop = total sale for the previous month

In [ ]:
df_BL = sales.copy()[['shop_id', 'item_id',
                      'date_block_num', 'total_monthly_sale']]

target = 'total_monthly_sale'
# Train-test split
x_train, y_train, x_test, y_test = get_train_test(df_BL, target, 12)


train_BL = pd.concat([x_train, y_train], axis=1)
train_BL = train_BL.drop_duplicates(subset=['shop_id', 'item_id'], keep='last')

test_BL = pd.concat([x_test, y_test], axis=1)
test_BL = pd.merge(test_BL, train_BL, how='left', on=[
    'shop_id', 'item_id']).dropna()


rmse_BLmodel = mean_squared_error(test_BL['total_monthly_sale_x'],
                                  test_BL['total_monthly_sale_y'], squared=False)
print('The root mean squared error of prediction with baseline model is:',
      rmse_BLmodel)

## Feature Engineering

In [ ]:
data = sales.copy()

# Mean-encode 'item_id' and 'shop_id'
target = 'total_monthly_sale'
data = mean_encode(data, 'shop_id', target)
data = mean_encode(data, 'item_id', target)


# Add 6-months lags of total_monthly_sale, monthly_sale_shop, monthly_sale_item
index_cols = ['shop_id', 'item_id', 'date_block_num']
lag_cols = ['total_monthly_sale', 'monthly_sale_shop', 'monthly_sale_item']
data = add_lag_features(data, lag_cols, index_cols, max_lag=6)


get_info(data)

## Create and Evaluate Models

In [ ]:
# Train-test split: Keep the last month for test

target = 'total_monthly_sale'
x_train, y_train, x_test, y_test = get_train_test(data, target, 12)


x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# Linear model

lr = make_pipeline(StandardScaler(), PCA(), LinearRegression())

crossval_lr = cross_val_score(lr, x_train, y_train,
                           cv=3, scoring='neg_mean_squared_error')

print('Root mean squared error with cross-validation:',
      sqrt(-1.0 * crossval_lr.mean()))

lr.fit(x_train, y_train)
pred_lr = lr.predict(x_test)
rmse_lr = mean_squared_error(y_test, pred_lr, squared=False)
print('The root mean squared error of prediction with linear regression is:', rmse_lr)

In [ ]:
# Light gradient boosting algorithm

lgbm = LGBMRegressor(feature_fraction=0.75,
                     min_data_in_leaf=2**5,
                     learning_rate=0.03,
                     bagging_seed=2**7,
                     num_leaves=2**7)

crossval_lgbm = cross_val_score(lgbm, x_train, y_train,
                                cv=3, scoring='neg_mean_squared_error')

print('Root mean squared error with cross-validation:',
      sqrt(-1.0 * crossval_lgbm.mean()))

lgbm.fit(x_train, y_train)
pred_lgbm = lgbm.predict(x_test)
rmse_lgbm = mean_squared_error(y_test, pred_lgbm, squared=False)
print(('The root mean squared error of prediction with '
      'light gradient boosting algorithm is:'), rmse_lgbm)

In [ ]:
# Get Feature Importances
importances = lgbm.feature_importances_

feature_importance_df = pd.DataFrame(
    importances, columns=['Feature Importance'], index=x_test.columns)

feature_importance_df.sort_values(
    by='Feature Importance', ascending=False, inplace=True)

# Feature importance plot
sns.set(style="whitegrid")
f, ax = plt.subplots(figsize=(15, 15))
sns.barplot(x='Feature Importance', y=feature_importance_df.index,
            data=feature_importance_df, color="b");

## Predict Future Sales

In [ ]:
get_info(test_data)

In [ ]:
test_data = test_data.drop('ID', axis=1)
test_data.sort_values(by=['shop_id', 'item_id'], inplace=True)
test_data = test_data.reset_index(drop=True)

get_info(test_data)

In [ ]:
# add features to test_data

lookup = data.copy().drop_duplicates(
    subset=['shop_id', 'item_id'], keep='last')
lookup.drop(['date_block_num', 'total_monthly_sale'], axis=1, inplace=True)

test = test_data.copy()
test = pd.merge(test, lookup, how='left',
                on=['shop_id', 'item_id']).fillna(0)

test['month'] = 11
test['year'] = 2015
test['season'] = 4

get_info(test)

In [ ]:
predictions = lgbm.predict(test)

In [ ]:
# Save results
with open('model.txt', 'w') as file:
    file.write(str(lgbm))


np.savetxt('predictions.csv', predictions, delimiter=',')

feature_importance_df.to_csv('feature_importance.csv')